In [ ]:
import os
import glob
import subprocess
import unidecode

import fiona
import geopandas as gpd
import pandas as pd

fiona.supported_drivers['KML'] = 'rw'

In [2]:
!pwd

/Users/federico/Downloads


In [ ]:
def run_command(cmd, cwd=None):
    subprocess.run(cmd, shell=True, check=True, cwd=cwd)


def run_otb_command(cmd, cwd=None):
    #otb_profile_path = os.getenv('OTBENV_PROFILE_PATH')
    otb_profile_path = '~/OTB-8.0.1-Linux64/otbenv.profile'
    if otb_profile_path:
        cmd = f"bash -c 'source {otb_profile_path}; {cmd}'"
    subprocess.run(cmd, shell=True, check=True, cwd=cwd)

def zonal_statistics(image_file, vector_file, out_vector_file):
    cmd = 'otbcli_ZonalStatistics ' \
        f'-in {image_file} ' \
        f'-inzone.vector.in {vector_file} ' \
        f'-out.vector.filename {out_vector_file} '
    run_otb_command(cmd)

In [ ]:
def download_from_bucket(bucket, path_dst):
    if not os.path.exists(path_dst):
        cmd = f"gsutil cp -n  {bucket} {path_dst}"
        subprocess.run(cmd, shell=True)
    else:
        print("File exists")

def upload_to_bucket(bucket, path_orig):
    if os.path.exists(path_orig):
        cmd = f"gsutil cp {path_orig} {bucket}"
        subprocess.run(cmd, shell=True)
    else:
        print("File doesn't exists")

def run_postprocess_raster(path_unet_results, lulc_masks, rendered_raster_path, thr, municipio_id):
    if not os.path.exists(rendered_raster_path):
        #filtro pixeles con probabilidad < threshold
        run_otb_command(f'otbcli_BandMath -il {path_unet_results} -out raster_tmp_math.tif -exp "(im1b1>={thr})*im1b1 + (im1b1<{thr})*0"')
        #alineo tif y lulc recortado
        run_otb_command(f"otbcli_Superimpose -inr raster_tmp_math.tif -inm {lulc_masks} -out {municipio_id}_lulc.tif")
        #filtro pixeles que no cae en el uso del suelo built-up
        run_otb_command(f'otbcli_BandMath -il raster_tmp_math.tif {municipio_id}_lulc.tif -out raster_tmp_math_2.tif -exp "((im2b1==1)*im1b1 + (im2b1==0)*0)"')
        #seteo no data y comprimo el raster
        run_otb_command(f'gdal_translate -a_nodata 0 raster_tmp_math_2.tif {municipio_id}_thr.tif -co "COMPRESS=LZW" -co "TILED=YES"')
        subprocess.run(f"python3 ./script/render_rgb.py {municipio_id}_thr.tif {rendered_raster_path}", shell=True)
        files_del = glob.glob("raster_tmp*.gpkg")
        for f in files_del:
            os.remove(f)
    else:
        print('File exists')
    
def downscale_grid(grid_path, level=17):
    gdf_grid = gpd.read_file(grid_path)
    gdf_grid['tile_level'] = gdf_grid['tile_id'].apply(lambda x: str(x)[:17])
    gdf_grid.dissolve(by='tile_level').to_file("grid_tmp_dissolve.gpkg")

def run_postprocess_grid(raster_path, grid_filt_name, thr, grid_downscaled_path="grid_tmp_dissolve.gpkg"):
    if not os.path.exists(grid_filt_name):
        zonal_statistics(raster_path, grid_downscaled_path, 'grid_tmp.gpkg')
        gdf_zs = gpd.read_file('grid_tmp.gpkg')
        gdf_zs_max = gdf_zs[['tile_level', 'max_0']].groupby('tile_level').median().reset_index(drop=False)
        gdf_zs = pd.merge(gdf_zs_max, gdf_zs[['tile_level', 'geometry']], on='tile_level')
        gdf_zs = gpd.GeoDataFrame(gdf_zs, geometry=gdf_zs.geometry)
        gdf_zs_filt = gdf_zs[gdf_zs["max_0"]>=thr]
        if gdf_zs_filt.shape[0]>0:
            gdf_zs_filt.to_file(grid_filt_name, driver='KML')
        else:
            print("Empty grid or probability map")
        files_del = glob.glob("grid_tmp*.gpkg")
        for f in files_del:
            os.remove(f)
    else:
        print('File exists')

Probability threshold (scaled 0..255)

In [ ]:
thr = 160

Load municipality geometry

In [ ]:
path_municipality = #INSERT GEOMETRIES PATH
gdf = gpd.read_file(path_municipality)
row = gdf.iloc[0]

In [ ]:
region = row['REGION']
province_id = row['DPTO_COD']
municipio_id = row['COD_MUNICIPIO']

In [ ]:
dir_results = f"resultados/{province_id}/{municipio_id}"
os.makedirs(dir_results, exist_ok=True)
#probability map final path
rendered_raster_path = os.path.join(dir_results, f"{municipio_id}.tif")
#results from unet path
path_unet_results = os.path.join(dir_results, f"{municipio_id}.tif")
#grid path
grid_path = os.path.join(dir_results,f"{municipio_id}.gpkg")
grid_filt_name = os.path.join(dir_results, f"{municipio_id}.kml")
#lulc path
mask_path = os.path.join(dir_results, f"{municipio_id}_mask.tif")

In case Google Cloud Platform (GCP) buckets needed to download raw data

In [ ]:
#Buckets paths
bucket_unet_result = f"gs://dym-temp-public/immap-dymaxion/results/{province_id}/{municipio_id}/{municipio_id}.tif"
bucket_mask = f"gs://dym-temp-public/immap-dymaxion/masks/{province_id}/{municipio_id}/{municipio_id}.tif"
bucket_grid = f"gs://dym-temp-public/immap-dymaxion/grids/{province_id}/{municipio_id}/{municipio_id}.gpkg"
bucket_kml = f"gs://dym-temp-public/immap-dymaxion/delivery/{province_id}/{municipio_id}/"
bucket_rendered = f"gs://dym-temp-public/immap-dymaxion/delivery/{province_id}/{municipio_id}/"

#Download data from GCP
download_from_bucket(bucket_unet_result, path_unet_results)
download_from_bucket(bucket_mask, mask_path)
download_from_bucket(bucket_grid, grid_path)

Performs postprocessing (raster and grid)

In [ ]:
run_postprocess_raster(path_unet_results, mask_path, rendered_raster_path, thr, municipio_id)
downscale_grid(grid_path)
run_postprocess_grid(f"{municipio_id}_thr.tif", grid_filt_name, thr)

Uploads results to GCP (if needed)

In [ ]:
upload_to_bucket(bucket_kml, grid_filt_name)
upload_to_bucket(bucket_rendered, rendered_raster_path)